Twee versie tabellen 

ANN
- (+) trailers
- (+) '_|'
- trailer bestaat uit 2 cijfers achter elkaar op voorwaarde dat je al een spatie hebt gezien op die regel, een verschil van 2
- zelfde geldt voor '_|' elke keer dar je dit verschijnsel tegenkomt is er een verschil van 2
i = 0
ann_ann = []
for line in file:
    lijn lezen tot na eerste spatie (versnummer en spatie)
    for character in line:
        i += 1
        if character in '[0-9]{2}|_\|': (regex inzetten: )
            ann_ann.append(i)

MiMi
- (-) trailers
- (+) hypens en plussen 
- bestaat ui 1 positie
i = 0
ann_mimi = []
for line in file:
    for character in line:
        i += 1
        if character in {+, -}:
            ann_mimi.append(i)
            

Aanpak
- Iteren over elke Psalm, haal ik karakters uit, sla de index van het karakter op
- 2 index lijsten maken: ann_ann_list, ann_mimi_list
- ann_ann samenvoegen met ann_mimi in position_dict. position_dict is per chapter, dan leegmaken
- vertalen van oud (ann) naar nieuw (mimi). 
- op alle posities waar er iets verspringt, in de nieuwe of oude file, heb je een entry in position_dict
- de verspringpunten zijn de keys; het aantal karakters dat het verschilt de values
- vb voor Psalm 1: index 7 : int(1) (h-isj), voor mimifile '-' +1 
                   index 14 : int(-1) asjer05, voor annfile '05' -2 
                   
- iaa mention dict vullen met aangepaste mention strings die indices van annotaties bevatten 'mention 1 5', begin index 1 en eind index 5 dus apart aanpassen
- mention vertaler functie: def translate_mention()
- binnen translate_mention(), functie: def translate_index() die begin en eind index aanpast, gebruik daar voor Reindex() uit declust.py

PS_A   = $(HOME)/Sites/brat/data/coref/Psalms/annotate
PS_B   = $(HOME)/github/cmerwich/participant-analysis/mimi

In [ ]:
__author__ = 'erwich/sikkel'

In [1]:
import getopt
import os, sys
from sys import argv, exit, stderr
import re
from pprint import pprint
from collections import OrderedDict
from operator import attrgetter

In [2]:
#pattern = re.compile('\&|[0-9]{2}|_\|')
# = re.compile('[0-9]{1,3}[\s]') # match verses

PATTERN = re.compile('[0-9]{2}|_\|')
TAB = '\t'
NL = '\n'

In [3]:
class Mention:
    def __init__(self, name, start=0, end=0, lex='', note='', file=0):
        self.name = name    # Identifier of the mention, e.g. T32
        self.start = start  # Start of the position in the file
        self.end = end      # End of the position in the file
        self.lex = lex      # Lexical information
        self.note = note    # AnnotatorNotes
        self.file = file    # Index of the file to which it is to be written

In [4]:
def error(*args, **kwargs):
    print(*args, file=stderr, **kwargs)
    exit(1)

In [5]:
def Usage():
    stderr.write('usage: declust -a input.ann input.txt...\n')
    exit(1)

In [6]:
def GetOffsetsAnn(chapter_text):
    
    i = 0
    ann_txt_list = []
    with open(chapter_text) as fh:
        for (ln, line) in enumerate(fh):
            match = PATTERN.finditer(line)
            for character in line:
                i += 1
                for m in match:
                    # filter out psalms title on line 0, 
                    # and (0, 2) for verse numbers with two digits or more
                    if ln != 0 and m.span() != (0, 2):
                        #ann_txt_list.append(f'{i+m.start()} {i+m.start()+1}') #{m.end()}
                        ann_txt_list.append(i+m.start())
                                          
    return ann_txt_list

In [7]:
def GetOffsetsMimi(chapter_text):
    
    i = 0
    mimi_txt_list = []
    
    with open(chapter_text) as fh:
        for (ln, line) in enumerate(fh):
            firstChar = line[0]
            for character in line: 
                #print(character, i)
                i += 1
                if character in {'+', '-'}:
                    mimi_txt_list.append(i-1)
               
    return mimi_txt_list         

In [8]:
def Merge(ann_txt_list, mimi_txt_list):
    d = {}
    for i in ann_txt_list:
        d[i] = -2
    for e in mimi_txt_list:
        d[e] = 1
    od = OrderedDict(sorted(d.items()))
    return od

In [9]:
def MakeOffsets(od):
    offset = 0
    jumps_list = []
    for k, v in od.items():
        point = k + (offset if v == 1 else 0)
        offset += v
        jumps_list.append((point, offset))
    return jumps_list

In [10]:
def find_index(jumps_list, coor):
    i = 0
    while jumps_list[i+1][0] < coor:
        i += 1
    return i

In [11]:
def TranslateIndex(jumps_list, mentions): 
    for m in mentions: 
        i = find_index(jumps_list, m.start)
        print('i: ', i, 't0 ', jumps_list[i][0], 't1 ', jumps_list[i][1])
        if m.start == jumps_list[i][0]:
            m.start = m.start + jumps_list[i][1]
            m.end = m.end + jumps_list[i][1]

In [12]:
def Parse(ann_file, jumps_list):
    '''
    Parses a plain text annotation file per Bible Book chapter 
    (e.g. Psalms 001) that has been annotated for coreference information.
    Returns a list of lists of coref-mention objects and 
    a list of mention objects. 
    '''
    
    errors = 0
    t2mDict = {}
    Mentions = []
    Corefs = []
    MentionIndexDict = {}
    mentionNote = {}
    dataPartsList = []

    firstChars = {'T', '#', '*'}
    cClass = 0

    with open(ann_file) as fh:
        for (i, line) in enumerate(fh):
            epos = f'{i + 1} '
            line = line.rstrip('\n')
            firstChar = line[0]

            if firstChar not in firstChars:
                error(f'{epos}Unrecognized line "{line}"')
                errors +=1
                continue

            numFields = 2 if firstChar =='*' else 3
            parts = line.split('\t')

            if len(parts) != numFields:
                error(f'{epos}line does not have exactly {numFields} parts: "{line}"')
                errors += 1
                continue

            if firstChar == 'T':
                (tPart, mentionStr, aWord) = parts
                mParts = mentionStr.split()
                if len(mParts) != 3:
                    error(f'{epos}T-line mention does not have exactly 3 parts: "{line}"')
                    errors += 1
                    continue
                (mm, aStart, aEnd) = mParts
                start = int(aStart)
                end = int(aEnd)
                # adjust mention start and end indices for iaa analysis 
                #theStart, theEnd = TranslateIndex(jumps_list, start, end)
                #theStart = TranslateIndex(jumps_list, start)
                #theEnd = TranslateIndex(jumps_list, end)
                t2mDict[tPart] = mentionStr
                m = Mention(tPart, start, end, aWord)
                MentionIndexDict[tPart] = len(Mentions)
                Mentions.append(m)
                
            elif firstChar == '*':
                corefSets = set()
                (char, data) = parts
                dataParts = data.split()
                if len(dataParts) <= 1 or dataParts[0] != 'Coreference':
                    error(f'{epos}*-line spec does not have the right parts: "{line}"')
                    errors += 1
                    continue
                cClass += 1
                dataPartsList.append(dataParts)
            
            elif firstChar == '#':
                (code, spec, note) = parts
                sParts = spec.split()
                if len(sParts) != 2:
                    error(f'{epos}#-line spec does not have exactly 2 parts: "{line}"')
                    errors += 1
                    continue
                tPart = sParts[1]
                if tPart in MentionIndexDict:
                    Mentions[MentionIndexDict[tPart]].note = note
                else:
                    error(f'Annotatornote "{note}": {tPart}: Mention not found')
                    sys.exit(1)
                
                mentionNote.setdefault(tPart, set()).add(note)
                   
        for l in dataPartsList:
            corefSets = set() # for error analysis 
            corefLists = []
            for tPart in l[1:]:
                if tPart in corefSets:
                    error(f'{epos}*-"{tPart} occurs in multiple classes "{corefSets[tPart]}" in "{line}"')
                    errors += 1
                    continue
                corefSets.add(t2mDict[tPart]) # for error analysis 
                if tPart in MentionIndexDict:
                    corefLists.append(Mentions[MentionIndexDict[tPart]])
                else:
                    error(f'Coref: {tPart}: Mention not found')
                    sys.exit(1)
            Corefs.append(corefLists) # list of lists, one list is one coreference class. 
            
    if errors:
        error(f'There are {errors} errors in annotation file(s)')
    
    return Mentions, Corefs

In [13]:
def OpenAnn(path):
    '''
    Opens an annotation file with extension `.ann' for each chapter
    in current folder. 
    '''
    
    filename_w_ext = os.path.basename(path)
    filename, file_extension = os.path.splitext(filename_w_ext)
    filename_ann = f'{filename}_n{file_extension}'
    ann_file = open(filename_ann, 'w')
    
    return ann_file

#OpenAnn('/Users/Christiaan/Sites/brat/data/coref/Psalms/annotate/Psalms_001.ann')

In [14]:
def WriteMentions(mentions, ann_file):
    '''
    Writes the mentions and annotatornotes on mentions to the corresponding ann file. 
    '''
    
    i = 0
    sorted_mentions = sorted(mentions, key=attrgetter('start'))
    for m in sorted_mentions:
        ann_file.write(f'{m.name}{TAB}Mention {str(m.start)} {str(m.end)}{TAB}{m.lex}{NL}')
        
        if m.note != '':
            i += 1
            ann_file.write(f'#{str(i)}{TAB}AnnotatorNotes {m.name}{TAB}{m.note}{NL}')

In [15]:
def WriteCorefs(corefs, ann_file):
    '''
    Writes a coref class with mentions to .ann file object.
    '''
    
    for c in corefs:
        ann_file.write(f'*{TAB}Coreference')
        sorted_corefs = sorted(c, key=attrgetter('start'))
        for m in sorted_corefs:
            ann_file.write(f' {m.name}') 
        ann_file.write('\n')

In [16]:
def Close(file): 
    '''
    Close ann file object.
    '''
    
    file.close()

In [17]:
def Translate(ann_txt, mimi_txt, ann_file):
    # ann_txt: path to ann txt files
    # mimi_txt = path to mimi txt files 
    # ann_file = path to (chris) ann_file
    
    ann_offsets = GetOffsetsAnn(ann_txt)
    mimi_offsets = GetOffsetsMimi(mimi_txt)
    ordered_dict = Merge(ann_offsets, mimi_offsets)
    jumps_list = MakeOffsets(ordered_dict)
    pprint(jumps_list)
    Mentions, Corefs = Parse(ann_file, jumps_list)
    TranslateIndex(jumps_list, Mentions)
    new_ann_file = OpenAnn(ann_file)
    WriteMentions(Mentions, new_ann_file)
    WriteCorefs(Corefs, new_ann_file)
    Close(new_ann_file)

In [18]:
Translate('Psalms_002-ann.txt', 'Psalms_002-mimi.txt', 'Psalms_002.ann')


[(28, 1),
 (43, -1),
 (55, -3),
 (61, -2),
 (89, -1),
 (98, 0),
 (102, -2),
 (119, -1),
 (125, 0),
 (137, 1),
 (145, 2),
 (146, 0),
 (157, 1),
 (159, 2),
 (182, 3),
 (181, 1),
 (199, 2),
 (205, 3),
 (209, 4),
 (214, 5),
 (217, 6),
 (223, 7),
 (220, 5),
 (229, 6),
 (238, 7),
 (253, 8),
 (257, 6),
 (269, 7),
 (300, 8),
 (305, 9),
 (318, 10),
 (311, 8),
 (327, 9),
 (341, 10),
 (346, 11),
 (362, 12),
 (367, 13),
 (358, 11),
 (371, 12),
 (390, 13),
 (395, 14),
 (407, 15),
 (394, 13),
 (420, 14),
 (429, 15),
 (433, 13),
 (479, 14),
 (470, 12),
 (485, 13),
 (493, 14),
 (492, 12),
 (519, 13),
 (540, 14),
 (548, 15),
 (538, 13),
 (564, 14)]
i:  0 t0  28 t1  1
i:  0 t0  28 t1  1
i:  0 t0  28 t1  1
i:  0 t0  28 t1  1
i:  0 t0  28 t1  1
i:  1 t0  43 t1  -1
i:  2 t0  55 t1  -3
i:  4 t0  89 t1  -1
i:  3 t0  61 t1  -2
i:  5 t0  98 t1  0
i:  6 t0  102 t1  -2
i:  9 t0  137 t1  1
i:  11 t0  146 t1  0
i:  11 t0  146 t1  0
i:  13 t0  159 t1  2
i:  13 t0  159 t1  2
i:  13 t0  159 t1  2
i:  13 t0  159 t1  2